#### Setup

In [ ]:
from specific import *

### Specify the experiments to compare

In [ ]:
experiments = ("15_most_important", "no_temporal_shifts")
assert len(experiments) == 2

### Load data

In [ ]:
import importlib.util

spec_modules = {}
spec_data = {}
spec_models = {}
for experiment in experiments:
    # Load the different experiments' modules.
    spec = importlib.util.spec_from_file_location(
        f"{experiment}_specific", str(PROJECT_DIR.parent / experiment / "specific.py")
    )
    spec_modules[experiment] = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(spec_modules[experiment])
    spec_data[experiment] = (
        endog_data,
        exog_data,
        master_mask,
        filled_datasets,
        masked_datasets,
        land_mask,
    ) = spec_modules[experiment].get_offset_data()
    spec_models[experiment] = spec_modules[experiment].cross_val_cache.load()[1]

### Check that the masks are aligned

In [ ]:
comp_masks = [spec_data[experiment][2] for experiment in experiments]
assert np.all(comp_masks[0] == comp_masks[1])

### Predict BA

In [ ]:
n_threads = get_ncpus()

master_mask = spec_data[experiments[0]][2]
target_ba = get_masked_array(endog_data.values, master_mask)

preds_cache = SimpleCache(f"predictions_{'_'.join(experiments)}", cache_dir=CACHE_DIR)

# XXX:
# preds_cache.clear()


@preds_cache
def get_predictions():
    predictions = {}

    for experiment in experiments:
        spec_models[experiment].n_jobs = n_threads
        with parallel_backend("threading", n_jobs=n_threads):
            predictions[experiment] = get_masked_array(
                spec_models[experiment].predict(spec_data[experiment][1]), master_mask
            )
    return predictions


predictions = get_predictions()

### Compare the predictions

In [ ]:
from wildfires.analysis import MidpointNormalize

model_name = f"comparison_{'_'.join(experiments)}"

diffs = predictions[experiments[1]] - predictions[experiments[0]]

# Plotting params.
figsize = (6, 3.2)
mpl.rcParams["figure.figsize"] = figsize

coast_linewidth = 0.5
date_str = "2010-01 to 2015-01"

v0 = abs(min(np.min(predictions[experiment]) for experiment in experiments))
v1 = max(np.max(predictions[experiment]) for experiment in experiments)

vmax = max((v0, v1))
vmin = -vmax

boundaries = [vmin, -1e-2, -1e-3, -1e-5, 0, 1e-5, 1e-3, 1e-2, vmax]

# Plotting predicted.
fig = cube_plotting(
    diffs,
    cmap="brewer_RdYlBu_11",
    label="Burned Area Fraction",
    title=f"BA Change (No Temporal - Top 15)\n{date_str}",
    log=True,
    coastline_kwargs={"linewidth": coast_linewidth},
    log_auto_bins=False,
    colorbar_kwargs={"norm": MidpointNormalize(midpoint=0)},
    #     min_edge=vmin,
    #     extend=None,
    #     boundaries=boundaries,
)
figure_saver.save_figure(fig, f"ba_{model_name}", sub_directory="predictions")

### Load all feature data

In [ ]:
experiment = "fire_seasonality_paper"
spec = importlib.util.spec_from_file_location(
    f"{experiment}_specific", str(PROJECT_DIR.parent / experiment / "specific.py")
)
spec_modules[experiment] = importlib.util.module_from_spec(spec)
spec.loader.exec_module(spec_modules[experiment])
spec_data[experiment] = (
    endog_data,
    exog_data,
    master_mask,
    filled_datasets,
    masked_datasets,
    land_mask,
) = spec_modules[experiment].get_offset_data()

### Correlations between diffs and other variables - virtually no correlation between the diffs and any of the features

In [ ]:
diff_data = get_unmasked(diffs)
all_features = spec_data["fire_seasonality_paper"][1].copy()
all_features["Diffs"] = diff_data

In [ ]:
with figure_saver("diff_corr_plot"):
    corr_plot(shorten_columns(all_features), fig_kwargs={"figsize": (12, 8)})